In [3]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import numpy as np

In [36]:
import csv

In [1]:
def con_cur_to_bgg_db():
    con = pg2.connect(host='34.216.22.202',
                  dbname='postgres',
                  user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

In [7]:
con, cur = con_cur_to_bgg_db()
cur.execute('SELECT * FROM dedupe d \
INNER JOIN test t \
ON d.uid = t.uid AND d.gid = t.gid \
WHERE gc > 1;')
results = cur.fetchall()
con.close()

In [8]:
df = pd.DataFrame(results)

In [10]:
df.sort_values('uid')

,gc,gid,rating,uid
34253,2,173346,9.5,7
34254,2,173346,9.0,7
22876,2,5206,8.0,24
22877,2,5206,6.0,24
11831,2,15512,6.0,24
706,2,92415,8.0,24
707,2,92415,7.0,24
11832,2,15512,5.0,24
981,2,553,7.0,38
13470,2,1797,5.0,38


In [11]:
37373/10957927

0.003410590342498175

In [13]:
grouped = df.groupby(['uid', 'gid']).mean()

In [18]:
grouped = grouped.drop('gc', axis=1)

In [23]:
include = grouped[grouped['rating'] >= 5.5]

In [24]:
len(include)

17074

In [25]:
include.head()

rating
uid gid           
7   173346    9.25
24  5206      7.00
    15512     5.50
    92415     7.50
38  553       7.25

In [28]:
include = include.round(2)

In [30]:
include.to_csv('../data/include')

In [41]:
with open('../data/include', 'r') as f:
    con, cur = con_cur_to_bgg_db()
    cur.copy_from(f, 'foo', sep=',')
    con.close()

In [37]:
with open('../data/include', 'r') as f:
    foo = f.read()

In [45]:
con, cur = con_cur_to_bgg_db()
cur.execute('INSERT INTO dedupe VALUES\
            {};'.format(", ".join(repr(e) for e in data)))
con.commit()
con.close()

In [42]:
with open('../data/include', 'r') as f:
    data=[tuple(line) for line in csv.reader(f)]